<a href="https://colab.research.google.com/github/RafaelCaballero/BME/blob/main/c%C3%B3digo/proyecto_4_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto 4 - Solución

Ejecutar el siguiente código para asegurar que las librerías están cargadas

In [1]:
modules = ["yfinance"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")
            try:
                r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)

Instalando módulos
yfinance  encontrado
¡Terminado!


Preparamos los datos

In [ ]:

import yfinance as yf
import pandas as pd

# Definir los símbolos de los 5 primeros valores del IBEX 35
symbols = ['SAN.MC', 'BBVA.MC', 'IBE.MC', 'ITX.MC', 'REP.MC']  # cód. Yahoo
nombre = ['Santander', 'BBVA', 'Iberdrola', 'Inditex', 'Repsol']  # Nombre

# descargamos y nos quedamos solo con los valores al cierre
df_raw = yf.download(tickers=symbols, period='max').loc[:,"Adj Close"]

# solo queremos los incrementos diarios y solo filas in nulos
df_raw = df_raw.dropna().pct_change().dropna()

# renombrar columna
df_raw.columns = [nombre[symbols.index(c)]+"_hoy" for c in df_raw.columns]
# día de la semana
df_raw['Día'] = pd.to_datetime(df_raw.index).day_name()
df_raw['Día'] = df_raw['Día'].replace({
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})
# semana del año
df_raw['Semana'] = df_raw.index.isocalendar().week


df_raw['BBVA_ayer'] = df_raw['BBVA_hoy'].shift(1)
df_raw['Iberdrola_ayer'] = df_raw['Iberdrola_hoy'].shift(1)
df_raw['Inditex_ayer'] = df_raw['Inditex_hoy'].shift(1)
df_raw['Repsol_ayer'] = df_raw['Repsol_hoy'].shift(1)
df_raw['Santander_ayer'] = df_raw['Santander_hoy'].shift(1)
df_raw['Santander_mañana'] = df_raw['Santander_hoy'].shift(-1)

df = df_raw.dropna()
df

[*********************100%***********************]  5 of 5 completed


,BBVA_hoy,Iberdrola_hoy,Inditex_hoy,Repsol_hoy,Santander_hoy,Día,Semana,BBVA_ayer,Iberdrola_ayer,Inditex_ayer,Repsol_ayer,Santander_ayer,Santander_mañana
Date,,,,,,,,,,,,,
2001-05-28 00:00:00+00:00,0.013333,0.000000,-0.006108,-0.005542,0.007279,Lunes,22,-0.018691,-0.022828,-0.001663,-0.006881,-0.016114,-0.009936
2001-05-29 00:00:00+00:00,0.002506,-0.009734,0.009497,-0.015328,-0.009936,Martes,22,0.013333,0.000000,-0.006108,-0.005542,0.007279,0.001825
2001-05-30 00:00:00+00:00,0.000625,-0.005243,0.001660,-0.004245,0.001825,Miércoles,22,0.002506,-0.009734,0.009497,-0.015328,-0.009936,0.024590
2001-05-31 00:00:00+00:00,0.005622,0.007905,0.013812,-0.002369,0.024590,Jueves,22,0.000625,-0.005243,0.001660,-0.004245,0.001825,-0.020444
2001-06-01 00:00:00+00:00,-0.007453,-0.009804,-0.002725,0.004274,-0.020444,Viernes,22,0.005622,0.007905,0.013812,-0.002369,0.024590,0.020871
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-11 00:00:00+00:00,0.006284,0.006652,0.010634,-0.002905,0.008269,Viernes,41,-0.004530,-0.017429,-0.002274,0.001246,0.000110,0.007326
2024-10-14 00:00:00+00:00,0.012059,0.015786,0.015408,-0.007074,0.007326,Lunes,42,0.006284,0.006652,0.010634,-0.002905,0.008269,0.006079
2024-10-15 00:00:00+00:00,0.004468,0.016986,0.001110,-0.020117,0.006079,Martes,42,0.012059,0.015786,0.015408,-0.007074,0.007326,-0.005179


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6007 entries, 2001-05-25 00:00:00+00:00 to 2024-10-18 00:00:00+00:00
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BBVA_hoy          6007 non-null   float64
 1   Iberdrola_hoy     6007 non-null   float64
 2   Inditex_hoy       6007 non-null   float64
 3   Repsol_hoy        6007 non-null   float64
 4   Santander_hoy     6007 non-null   float64
 5   Día               6007 non-null   object 
 6   Semana            6007 non-null   UInt32 
 7   BBVA_ayer         6006 non-null   float64
 8   Iberdrola_ayer    6006 non-null   float64
 9   Inditex_ayer      6006 non-null   float64
 10  Repsol_ayer       6006 non-null   float64
 11  Santander_ayer    6006 non-null   float64
 12  Santander_mañana  6006 non-null   float64
dtypes: UInt32(1), float64(11), object(1)
memory usage: 639.4+ KB


In [ ]:
df_raw.describe()

,BBVA_hoy,Iberdrola_hoy,Inditex_hoy,Repsol_hoy,Santander_hoy,Semana,BBVA_ayer,Iberdrola_ayer,Inditex_ayer,Repsol_ayer,Santander_ayer,Santander_mañana
count,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.0,6006.000000,6006.000000,6006.000000,6006.000000,6006.000000,6006.000000
mean,0.000337,0.000500,-0.000193,0.000283,0.000321,26.67438,0.000334,0.000501,-0.000190,0.000283,0.000319,0.000324
std,0.021670,0.015385,0.088107,0.019282,0.021999,14.888335,0.021670,0.015387,0.088115,0.019284,0.022000,0.021999
min,-0.161793,-0.140628,-6.648325,-0.157542,-0.198864,1.0,-0.161793,-0.140628,-6.648325,-0.157542,-0.198864,-0.198864
25%,-0.010264,-0.006364,-0.008198,-0.008967,-0.010544,14.0,-0.010264,-0.006365,-0.008187,-0.008967,-0.010550,-0.010529
50%,0.000000,0.000416,0.000000,0.000000,0.000141,27.0,0.000000,0.000417,0.000000,0.000000,0.000139,0.000148
75%,0.010630,0.007384,0.009633,0.009748,0.010682,39.0,0.010618,0.007386,0.009635,0.009750,0.010679,0.010683
max,0.220271,0.166881,0.460041,0.182259,0.232166,53.0,0.220271,0.166881,0.460041,0.182259,0.232166,0.232166


Cada fila representa un día. Tiene valores de cierre de hoy (hoy --> fecha) de Iberdrola, Inditex, BBVA y Santander, los valores que tuvieron ayer, y el BBVA de mañana, que es la etiqueta que queremos predecir.

a) Se pide hacer un modelo de regresión lineal para predecir `BBVA_mañana`. Se puede probar con varias combinaciones de columnas para intentar predecir `BBVA_mañana`...salvo el propio `BBVA_mañana` claro.

No hay que hacer demasiado, probar con 2 o 3 combinaciones y dar el margen del error.

b) Probar a hacer alguna predicción con el mejor modelo del apartado a; inventarse algunos valores para las columnas seleccionadas y ver qué valor predice.

In [ ]:
df_raw.columns

Index(['BBVA_hoy', 'Iberdrola_hoy', 'Inditex_hoy', 'Repsol_hoy',
       'Santander_hoy', 'Día', 'Semana', 'BBVA_ayer', 'Iberdrola_ayer',
       'Inditex_ayer', 'Repsol_ayer', 'Santander_ayer', 'Santander_mañana'],
      dtype='object')

In [ ]:
# Paso 1
yColumn = "Santander_mañana"

XColumns = ['BBVA_hoy', 'Iberdrola_hoy', 'Inditex_hoy', 'Repsol_hoy',
       'Santander_hoy',  'Semana', 'BBVA_ayer', 'Iberdrola_ayer',
       'Inditex_ayer', 'Repsol_ayer', 'Santander_ayer']

X = df[XColumns]
y = df[yColumn]

# paso 2
from sklearn.model_selection import train_test_split

test = 0.40 # 40%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# paso 3
from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train)

# paso 4
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}" )

r^2: 0.0 RMSE: 0.022, MAE:0.015


In [ ]:
modelo.predict([ [0,0,0,0,0,0,0,0,0,0,0] ])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([-0.00046495])

In [ ]:
modelo.predict([ [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1] ])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([-0.00546452])

Como es tan lamentable, voy a utiliza otros métodos de regresión

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Paso 1
yColumn = "Santander_mañana"

XColumns = ['BBVA_hoy', 'Iberdrola_hoy', 'Inditex_hoy', 'Repsol_hoy',
       'Santander_hoy',  'Semana', 'BBVA_ayer', 'Iberdrola_ayer',
       'Inditex_ayer', 'Repsol_ayer', 'Santander_ayer']

X = df[XColumns]
y = df[yColumn]

# paso 2
from sklearn.model_selection import train_test_split

test = 0.40 # 40%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# paso 3

metodo = RandomForestRegressor()
modelo = metodo.fit(X_train,y_train)

# paso 4
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}" )


r^2: -0.065 RMSE: 0.023, MAE:0.016


Siempre comparar con un método sencillo, el método base. en este caso considerar que el valor de SAN mañana es SAN hoy

In [ ]:
y_pred = X_test["Santander_hoy"]
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}" )

r^2: -0.9 RMSE: 0.03, MAE:0.022


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BBVA_hoy,6005.0,0.000337,0.021671,-0.161793,-0.010262,0.0,0.010626,0.220272
Iberdrola_hoy,6005.0,0.000505,0.015385,-0.140628,-0.006362,0.000419,0.007387,0.166881
Inditex_hoy,6005.0,-0.00019,0.088122,-6.648324,-0.008191,0.0,0.009638,0.460042
Repsol_hoy,6005.0,0.000284,0.019285,-0.157541,-0.008967,0.0,0.009753,0.182259
Santander_hoy,6005.0,0.000322,0.022001,-0.198864,-0.01053,0.000141,0.01068,0.232167
Semana,6005.0,26.672773,14.889321,1.0,14.0,27.0,39.0,53.0
BBVA_ayer,6005.0,0.000337,0.02167,-0.161793,-0.010262,0.0,0.010626,0.220272
Iberdrola_ayer,6005.0,0.000502,0.015388,-0.140628,-0.006362,0.000419,0.007387,0.166881
Inditex_ayer,6005.0,-0.000189,0.088122,-6.648324,-0.008191,0.0,0.009638,0.460042
Repsol_ayer,6005.0,0.000283,0.019286,-0.157541,-0.008967,0.0,0.009753,0.182259


In [ ]:
# Paso 1
yColumn = "Santander_mañana"

XColumns = [
       'Santander_hoy',  'Santander_ayer']

X = df[XColumns]
y = df[yColumn]

# paso 2
from sklearn.model_selection import train_test_split

test = 0.40 # 40%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# paso 3
#from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train)

# paso 4
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}" )

r^2: -0.006 RMSE: 0.022, MAE:0.015


RMSE: 0.022, MAE:0.015